In [ ]:
#title 0-  Instalar Elasticsearch y cliente Python
!pip install elasticsearch==8.11.0 -q
!pip install selenium webdriver-manager pdfplumber elasticsearch==8.11.0 -q


In [ ]:
import os
import json
import requests
from datetime import datetime
from elasticsearch import Elasticsearch
import time
import pdfplumber
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from elasticsearch import Elasticsearch, helpers


In [ ]:
#@title  1- Iniciar, configurar elasic
CLOUD_URL ="https://dbc32416c0ef45949b8ccbffa25c5a29.us-central1.gcp.cloud.es.io:443"
API_KEY = "RWZfODI1b0JtTUo5M0RpakplUDI6S2pEY19DajFJSHZaOS1nc1hSM3JvZw=="


In [ ]:
#@title 2- Crear cliente de ElasticSearch
es = Elasticsearch(
    CLOUD_URL,
    api_key=API_KEY,
    verify_certs=True
)
# Verificar conexión
if es.ping():
    print("Conexión exitosa a ElasticSearch Cloud")
    print(f"Cluster info: {es.info()['version']['number']}")
else:
    print("Error al conectar con ElasticSearch")

Conexión exitosa a ElasticSearch Cloud
Cluster info: 9.2.1


In [ ]:
#@title 3- Crear un index
INDEX_NAME = "index_proyecto"
# Eliminar index si existe (para empezar limpio)
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' eliminado")

Index 'index_proyecto' eliminado


In [ ]:
#@title 4- mapping
mapping = {
    "mappings": {
        "properties": {
            "titulo": {"type": "text"},
            "materia": {"type": "keyword"},
            "tipo_documento": {"type": "keyword"},
            "url_origen": {"type": "keyword"},
            "fecha_extraccion": {"type": "date"},
            "paginas": {"type": "integer"},
            "texto_completo": {"type": "text", "analyzer": "spanish"}
        }
    }
}

# Define el nombre del index  mapping
INDEX_NAME_FOR_MAPPING = "index_proyecto"

# Eliminar index si existe (para empezar limpio)
if es.indices.exists(index=INDEX_NAME_FOR_MAPPING):
    es.indices.delete(index=INDEX_NAME_FOR_MAPPING)
    print(f"Index '{INDEX_NAME_FOR_MAPPING}' eliminado antes de recrear")

es.indices.create(index=INDEX_NAME_FOR_MAPPING, body=mapping)
print(f"Índice '{INDEX_NAME_FOR_MAPPING}' creado con éxito")

Index 'index_proyecto' eliminado antes de recrear
Índice 'index_proyecto' creado con éxito


In [ ]:
#@title 5  primer webScraping
#Install Chrome
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# Ocultar webdriver
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {"source": "Object.defineProperty(navigator,'webdriver',{get:()=>false});"}
)

OK
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt

{'identifier': '2'}

In [ ]:
def scroll(driver):
    last = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        new = driver.execute_script("return document.body.scrollHeight")
        if new == last:
            break
        last = new


In [ ]:
import re

def extraer_links(urls):
    pdf_links = set()

    for url in urls:
        print("\n--- Visitando:", url)
        driver.get(url)
        time.sleep(5)
        scroll(driver)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # A tag
        for a in soup.find_all("a", href=True):
            link = a["href"]
            if ".pdf" in link.lower() or "drive.google.com" in link.lower():
                pdf_links.add(link)

        # onclick
        for l in re.findall(r'"([^"\\]+\\.pdf)"', html):
            pdf_links.add(l)

        # iframe
        for iframe in soup.find_all("iframe"):
            src = iframe.get("src", "")
            if ".pdf" in src:
                pdf_links.add(src)

    return list(pdf_links)

In [ ]:
def descargar_pdf(url, carpeta="pdfs"):
    os.makedirs(carpeta, exist_ok=True)

    filename = url.split("/")[-1].replace("?", "_")
    ruta = os.path.join(carpeta, filename)

    # Si es Google Drive → transformar URL
    if "drive.google.com" in url:
        if "uc?export=download" not in url:
            try:
                file_id = url.split("/d/")[1].split("/")[0]
                url = f"https://drive.google.com/uc?export=download&id={file_id}"
            except:
                print("URL de Drive no transformable:", url)
                return None

    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            with open(ruta, "wb") as f:
                f.write(r.content)
            return ruta
        else:
            print("❌ Error descargando:", url, " Status:", r.status_code)
            return None
    except:
        print("❌ Error descargando:", url)
        return None


In [ ]:
def extraer_texto_pdf(ruta):
    try:
        with pdfplumber.open(ruta) as pdf:
            texto = "\n".join([p.extract_text() or "" for p in pdf.pages])
            return texto, len(pdf.pages)
    except:
        return "", 0


In [ ]:
def subir_a_elastic(documentos):
    helpers.bulk(
        es,
        documentos,
        index=INDEX_NAME
    )
    print(f"✔ {len(documentos)} documentos insertados en Elastic")


In [ ]:
urls = [
    "https://altopuntaje.com/prueba-icfes-preguntas-saber-11-examenes/"
]

# 1) SCRAPING DE LINKS
links = extraer_links(urls)
print("\nPDFs encontrados:", len(links))

docs = []

# 2) DESCARGA + EXTRACCIÓN + INDEXACIÓN
for link in links:
    print("\nDescargando:", link)
    ruta = descargar_pdf(link)

    if not ruta:
        continue

    texto, paginas = extraer_texto_pdf(ruta)

    doc = {
        "titulo": os.path.basename(ruta),
        "materia": "ICFES",
        "tipo_documento": "pdf",
        "url_origen": link,
        "fecha_extraccion": datetime.utcnow(),
        "paginas": paginas,
        "texto_completo": texto,
    }

    docs.append(doc)

# 3) SUBIR A ELASTIC
if docs:
    subir_a_elastic(docs)
else:
    print("❌ No hay documentos para indexar")

driver.quit()



--- Visitando: https://altopuntaje.com/prueba-icfes-preguntas-saber-11-examenes/

PDFs encontrados: 32

Descargando: https://drive.google.com/uc?export=download&id=132bIxoHLkKFB-XooeV3OMgC4JWpgQ6TX


/tmp/ipython-input-3651119207.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "fecha_extraccion": datetime.utcnow(),



Descargando: https://drive.google.com/uc?export=download&id=16SngXFBe_NYjhwNa92f0Lxh0HBhTCehg

Descargando: https://drive.google.com/uc?export=download&id=191ALh7B3IJBbYeKhH_ieoNEVv8w23T0r

Descargando: https://drive.google.com/uc?export=download&id=10I5VDAXuVaYBSkW4OhkY2kBduoreNiw8

Descargando: https://drive.google.com/uc?export=download&id=1kkie7x9r-Eza6qtJ3e_zT-j2jkoe7k6i



Descargando: https://drive.google.com/uc?export=download&id=1ynVf3ETxqgDWy7ic8VvQP3ZZVlgtoB49

Descargando: https://drive.google.com/uc?export=download&id=1EjqsXdHADBMqZzmlXj72QO_7OETIvV37

Descargando: https://drive.google.com/uc?export=download&id=1lPdkCLJU9UooSIUNaRXjg2KRb6__Kn3J

Descargando: https://drive.google.com/uc?export=download&id=1LoykKkFIkEFxslBweOU7Y1ghGURyqqNv
❌ Error descargando: https://drive.google.com/uc?export=download&id=1LoykKkFIkEFxslBweOU7Y1ghGURyqqNv  Status: 404

Descargando: https://drive.google.com/uc?export=download&id=1SaR0-SPlJ5ffAHD4HNGkHQr88FSmXsWi



Descargando: https://drive.google.com/uc?export=download&id=1t-Syc6y3_WQI77zy21vUTQ2ZrTOuXyTD



Descargando: https://drive.google.com/uc?export=download&id=1x2Dvt5uNtW2hiaCwV3MF__bTSeWa9sSj



Descargando: https://drive.google.com/uc?export=download&id=1gAHomIpCx7dOPjWBChrMVZiu03X43uYp

Descargando: https://drive.google.com/uc?export=download&id=1LE1pisEb6v9WHlm2nB5bulQHNKLNwqoL

Descargando: https://drive.google.com/uc?export=download&id=1JYSFYIP54xVjbQCfScPmfeIhVjsneGuh

Descargando: https://drive.google.com/uc?export=download&id=1lHz3dToJXDDFK-qdaSvPRrSZdTvw-lMD

Descargando: https://drive.google.com/uc?export=download&id=1eMcIxKYZtZhCFMCJNxY_tpjwVO-a28jS

Descargando: https://drive.google.com/uc?export=download&id=1YyL2rPxMjtqjwb8eyOSqNwROA4OpDDPk

Descargando: https://drive.google.com/uc?export=download&id=1vKuqWBaSbNKSpVwYy5Tg-AXOWHJ1EXOL

Descargando: https://drive.google.com/uc?export=download&id=10HG_5jTquH18DPIedM2lE-zKR3xs2N1W

Descargando: https://drive.google.com/uc?export=download&id=1g9gRYulRwT4wBwwyQ-jvANe3Honad-GP

Descargando: https://drive.google.com/uc?export=download&id=13UA6VmIXXsQVZulRINKjN6264Jcl4y2I

Descargando: https://drive.google.com/uc?export=d


Descargando: https://drive.google.com/uc?export=download&id=1gVYxt9op6anLJz3S2MAgrDm9he63D8gI

Descargando: https://drive.google.com/uc?export=download&id=1F8s1stOoXEIBgJiP46OgCAgsmU-6k_ih

Descargando: https://drive.google.com/uc?export=download&id=1ZMH5ZDHmsHDwhmqlTwyyAHwuJX8WZ2jB

Descargando: https://drive.google.com/uc?export=download&id=1r6zX6U-Tign8i9rhcpovdGAWZCX9Y7eC

Descargando: https://drive.google.com/uc?export=download&id=1NljO0JrW_BTPbITjqhzxFwHCs4GB8rJN

Descargando: https://drive.google.com/uc?export=download&id=1MYTFjQJspVkbaxdPV8QudTmctWJvlZhs
✔ 31 documentos insertados en Elastic


In [ ]:
es.count(index="index_proyecto")

ObjectApiResponse({'count': 31, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
#@title 6- segundo webScraping
!pip install webdriver-manager # Install webdriver_manager to automatically handle ChromeDriver

import requests
from bs4 import BeautifulSoup

#@title Web scraping con Selenium - ICFES
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager # Import ChromeDriverManager
import time

# --- Configuración del navegador ---
# Install Google Chrome browser
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable

chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecutar sin abrir ventana
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36"
)

# Use ChromeDriverManager to automatically download and manage chromedriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# --- Abrir página ---
url = "https://www.icfes.gov.co/caja-de-herramientas-saber-11/practica/"
driver.get(url)
time.sleep(3)  # Esperar a que cargue el contenido

# --- Extraer links ---
links = driver.find_elements(By.TAG_NAME, "a")
for link in links:
    href = link.get_attribute("href")
    texto = link.text.strip()
    if href and ".pdf" in href.lower():  # Solo PDFs
        print(texto, href)

driver.quit()


OK
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 2s (73.9 kB/s)
Reading package lists... Done
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/source

In [ ]:
#@title 6.1- Lista final de PDFs con nombres limpios
pdfs = {
    "01_EXPLICADAS_LECTURA.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/01_EXPLICADAS_LECTURA.pdf",
    "02_PRACTICA_LECTURA.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/02_PRACTICA_LECTURA.pdf",
    "03_EXPLICADAS_MATEMATICAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/03_EXPLICADAS_MATEMATICAS.pdf",
    "04_PRACTICA_MATEMATICAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/04_PRACTICA_MATEMATICAS.pdf",
    "05_EXPLICADAS_SOCIALES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/05_EXPLICADAS_SOCIALES.pdf",
    "06_PRACTICA_SOCIALES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/06_PRACTICA_SOCIALES.pdf",
    "07_EXPLICADAS_CIENCIAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/07_EXPLICADAS_CIENCIAS.pdf",
    "08_PRACTICA_CIENCIAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/08_PRACTICA_CIENCIAS.pdf",
    "09_EXPLICADAS_INGLES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/09_EXPLICADAS_INGLES.pdf",
    "10_PRACTICA_INGLES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/10_PRACTICA_INGLES.pdf",
}

# Carpeta destino
os.makedirs("icfes_pdfs", exist_ok=True)

for filename, url in pdfs.items():
    print(f"Descargando: {filename} ...")
    resp = requests.get(url)

    if resp.status_code == 200:
        with open(f"icfes_pdfs/{filename}", "wb") as f:
            f.write(resp.content)
        print(f"✔ Guardado: {filename}")
    else:
        print(f"❌ Error al descargar {filename} — código {resp.status_code}")


Descargando: 01_EXPLICADAS_LECTURA.pdf ...
❌ Error al descargar 01_EXPLICADAS_LECTURA.pdf — código 406
Descargando: 02_PRACTICA_LECTURA.pdf ...
❌ Error al descargar 02_PRACTICA_LECTURA.pdf — código 406
Descargando: 03_EXPLICADAS_MATEMATICAS.pdf ...
❌ Error al descargar 03_EXPLICADAS_MATEMATICAS.pdf — código 406
Descargando: 04_PRACTICA_MATEMATICAS.pdf ...
❌ Error al descargar 04_PRACTICA_MATEMATICAS.pdf — código 406
Descargando: 05_EXPLICADAS_SOCIALES.pdf ...
❌ Error al descargar 05_EXPLICADAS_SOCIALES.pdf — código 406
Descargando: 06_PRACTICA_SOCIALES.pdf ...
❌ Error al descargar 06_PRACTICA_SOCIALES.pdf — código 406
Descargando: 07_EXPLICADAS_CIENCIAS.pdf ...
❌ Error al descargar 07_EXPLICADAS_CIENCIAS.pdf — código 406
Descargando: 08_PRACTICA_CIENCIAS.pdf ...
❌ Error al descargar 08_PRACTICA_CIENCIAS.pdf — código 406
Descargando: 09_EXPLICADAS_INGLES.pdf ...
❌ Error al descargar 09_EXPLICADAS_INGLES.pdf — código 406
Descargando: 10_PRACTICA_INGLES.pdf ...
❌ Error al descargar 10_PRA

In [ ]:
# Headers "reales" para que el servidor no devuelva 406
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,"
              "image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "es-ES,es;q=0.9",
    "Referer": "https://www.icfes.gov.co/",
    "Connection": "keep-alive",
}

pdfs = {
    "01_EXPLICADAS_LECTURA.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/01_EXPLICADAS_LECTURA.pdf",
    "02_PRACTICA_LECTURA.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/02_PRACTICA_LECTURA.pdf",
    "03_EXPLICADAS_MATEMATICAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/03_EXPLICADAS_MATEMATICAS.pdf",
    "04_PRACTICA_MATEMATICAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/04_PRACTICA_MATEMATICAS.pdf",
    "05_EXPLICADAS_SOCIALES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/05_EXPLICADAS_SOCIALES.pdf",
    "06_PRACTICA_SOCIALES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/06_PRACTICA_SOCIALES.pdf",
    "07_EXPLICADAS_CIENCIAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/07_EXPLICADAS_CIENCIAS.pdf",
    "08_PRACTICA_CIENCIAS.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/08_PRACTICA_CIENCIAS.pdf",
    "09_EXPLICADAS_INGLES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/09_EXPLICADAS_INGLES.pdf",
    "10_PRACTICA_INGLES.pdf": "http://icfes.acendra.com.co/wp-content/uploads/2024/12/10_PRACTICA_INGLES.pdf",
}

os.makedirs("icfes_pdfs", exist_ok=True)

for filename, url in pdfs.items():
    print(f"Descargando: {filename} ...")
    resp = requests.get(url, headers=headers)

    if resp.status_code == 200:
        with open(f"icfes_pdfs/{filename}", "wb") as f:
            f.write(resp.content)
        print(f"✔ Guardado: {filename}")
    else:
        print(f"❌ Error al descargar {filename} — código {resp.status_code}")


Descargando: 01_EXPLICADAS_LECTURA.pdf ...
✔ Guardado: 01_EXPLICADAS_LECTURA.pdf
Descargando: 02_PRACTICA_LECTURA.pdf ...
✔ Guardado: 02_PRACTICA_LECTURA.pdf
Descargando: 03_EXPLICADAS_MATEMATICAS.pdf ...
✔ Guardado: 03_EXPLICADAS_MATEMATICAS.pdf
Descargando: 04_PRACTICA_MATEMATICAS.pdf ...
✔ Guardado: 04_PRACTICA_MATEMATICAS.pdf
Descargando: 05_EXPLICADAS_SOCIALES.pdf ...
✔ Guardado: 05_EXPLICADAS_SOCIALES.pdf
Descargando: 06_PRACTICA_SOCIALES.pdf ...
✔ Guardado: 06_PRACTICA_SOCIALES.pdf
Descargando: 07_EXPLICADAS_CIENCIAS.pdf ...
✔ Guardado: 07_EXPLICADAS_CIENCIAS.pdf
Descargando: 08_PRACTICA_CIENCIAS.pdf ...
✔ Guardado: 08_PRACTICA_CIENCIAS.pdf
Descargando: 09_EXPLICADAS_INGLES.pdf ...
✔ Guardado: 09_EXPLICADAS_INGLES.pdf
Descargando: 10_PRACTICA_INGLES.pdf ...
✔ Guardado: 10_PRACTICA_INGLES.pdf


In [ ]:
!pip install pdfplumber

import pdfplumber
import os

carpeta = "icfes_pdfs"

for pdf_file in os.listdir(carpeta):
    if pdf_file.endswith(".pdf"):
        ruta = os.path.join(carpeta, pdf_file)
        print(f"Procesando {pdf_file}...")

        with pdfplumber.open(ruta) as pdf:
            texto = ""
            for page in pdf.pages:
                texto += page.extract_text() + "\n"

        # Guardar txt
        with open(ruta.replace(".pdf", ".txt"), "w", encoding="utf-8") as f:
            f.write(texto)

        print(f"✔ Extraído: {pdf_file.replace('.pdf', '.txt')}")

Procesando 09_EXPLICADAS_INGLES.pdf...
✔ Extraído: 09_EXPLICADAS_INGLES.txt
Procesando 08_PRACTICA_CIENCIAS.pdf...
✔ Extraído: 08_PRACTICA_CIENCIAS.txt
Procesando 10_PRACTICA_INGLES.pdf...
✔ Extraído: 10_PRACTICA_INGLES.txt
Procesando 03_EXPLICADAS_MATEMATICAS.pdf...
✔ Extraído: 03_EXPLICADAS_MATEMATICAS.txt
Procesando 07_EXPLICADAS_CIENCIAS.pdf...
✔ Extraído: 07_EXPLICADAS_CIENCIAS.txt
Procesando 05_EXPLICADAS_SOCIALES.pdf...
✔ Extraído: 05_EXPLICADAS_SOCIALES.txt
Procesando 06_PRACTICA_SOCIALES.pdf...
✔ Extraído: 06_PRACTICA_SOCIALES.txt
Procesando 02_PRACTICA_LECTURA.pdf...
✔ Extraído: 02_PRACTICA_LECTURA.txt
Procesando 01_EXPLICADAS_LECTURA.pdf...
✔ Extraído: 01_EXPLICADAS_LECTURA.txt
Procesando 04_PRACTICA_MATEMATICAS.pdf...
✔ Extraído: 04_PRACTICA_MATEMATICAS.txt


In [ ]:
es = Elasticsearch(
    CLOUD_URL,
    api_key=API_KEY,
    verify_certs=True
)

print("Ping:", es.ping())

Ping: True


In [ ]:
carpeta = "icfes_pdfs"

for file in os.listdir(carpeta):
    if file.endswith(".txt"):
        ruta = os.path.join(carpeta, file)

        with open(ruta, "r", encoding="utf-8") as f:
            contenido = f.read()

        doc = {
            "filename": file,
            "content": contenido,
        }

        es.index(index=INDEX_NAME , document=doc)
        print(f"📤 Subido: {file}")

📤 Subido: 04_PRACTICA_MATEMATICAS.txt
📤 Subido: 01_EXPLICADAS_LECTURA.txt
📤 Subido: 10_PRACTICA_INGLES.txt
📤 Subido: 09_EXPLICADAS_INGLES.txt
📤 Subido: 08_PRACTICA_CIENCIAS.txt
📤 Subido: 03_EXPLICADAS_MATEMATICAS.txt
📤 Subido: 06_PRACTICA_SOCIALES.txt
📤 Subido: 07_EXPLICADAS_CIENCIAS.txt
📤 Subido: 02_PRACTICA_LECTURA.txt
📤 Subido: 05_EXPLICADAS_SOCIALES.txt
